In [1]:
# Download Training dataset file.

import requests
URL = "https://www.dropbox.com/s/cem9ea35wllfsqg/train.csv?dl=1"
response = requests.get(URL)
open("train.csv", "wb").write(response.content)

200589097

In [2]:
# Download Testing dataset file.

URL = "https://www.dropbox.com/s/hrqzwaihgeibx4x/test.csv?dl=1"
response = requests.get(URL)
open("test.csv", "wb").write(response.content)

70794289

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from math import radians, cos, sin, asin, sqrt
from pyspark.sql.types import FloatType

spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()
df = spark.read.csv("./train.csv", header=True)
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- trip_duration: string (nullable = true)



In [4]:
df.show()

+---------+---------+-------------------+-------------------+---------------+-------------------+------------------+-------------------+------------------+------------------+-------------+
|       id|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|   pickup_longitude|   pickup_latitude|  dropoff_longitude|  dropoff_latitude|store_and_fwd_flag|trip_duration|
+---------+---------+-------------------+-------------------+---------------+-------------------+------------------+-------------------+------------------+------------------+-------------+
|id2875421|        2|2016-03-14 17:24:55|2016-03-14 17:32:30|              1|-73.982154846191406|40.767936706542969|-73.964630126953125|40.765602111816406|                 N|          455|
|id2377394|        1|2016-06-12 00:43:35|2016-06-12 00:54:38|              1|-73.980415344238281|40.738563537597656|-73.999481201171875|40.731151580810547|                 N|          663|
|id3858529|        2|2016-01-19 11:35:24|2016-01-19 12:

In [5]:
# Check for NaN or null values

df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+---------+---------------+----------------+---------------+----------------+---------------+-----------------+----------------+------------------+-------------+
| id|vendor_id|pickup_datetime|dropoff_datetime|passenger_count|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|store_and_fwd_flag|trip_duration|
+---+---------+---------------+----------------+---------------+----------------+---------------+-----------------+----------------+------------------+-------------+
|  0|        0|              0|               0|              0|               0|              0|                0|               0|                 0|            0|
+---+---------+---------------+----------------+---------------+----------------+---------------+-----------------+----------------+------------------+-------------+



In [6]:
# Removing id from `id` column.

df = df.withColumn("id", expr("replace(id, 'id', '')"))

In [7]:
# Covert Datatype of the dataframe.

def convert_datatype(df):
    df = df.withColumn("id",df.id.cast('int'))
    df = df.withColumn("vendor_id", df.vendor_id.cast('int'))
    df = df.withColumn("pickup_datetime", to_timestamp(df.pickup_datetime, "yyyy-MM-dd HH:mm:ss"))
    df = df.withColumn("dropoff_datetime", to_timestamp(df.dropoff_datetime, "yyyy-MM-dd HH:mm:ss"))
    df = df.withColumn("passenger_count",df.passenger_count.cast('int'))
    df = df.withColumn("pickup_longitude",df.pickup_longitude.cast('double'))
    df = df.withColumn("pickup_latitude",df.pickup_latitude.cast('double'))
    df = df.withColumn("dropoff_longitude",df.dropoff_longitude.cast('double'))
    df = df.withColumn("dropoff_latitude",df.dropoff_latitude.cast('double'))
    df = df.withColumn('store_and_fwd_flag', F.when(df.store_and_fwd_flag == 'N', 0).otherwise(1))
    df = df.withColumn("trip_duration",df.trip_duration.cast('int'))
    
    return df

In [8]:
# Call Convert datatype

df = convert_datatype(df)

In [9]:
# Check for columns data types

for col in df.dtypes:
    print(col[0]+" , "+col[1])

id , int
vendor_id , int
pickup_datetime , timestamp
dropoff_datetime , timestamp
passenger_count , int
pickup_longitude , double
pickup_latitude , double
dropoff_longitude , double
dropoff_latitude , double
store_and_fwd_flag , int
trip_duration , int


In [10]:
# Calculate difference in dropoff and pickup time

df = df.withColumn('diff_in_dropff_and_pickup',df["dropoff_datetime"].cast("double") - df['pickup_datetime'].cast("double"))

In [11]:
# Calculate the difference between trip duration and the difference of the pickup and dropoff time

df = df.withColumn('diff_in_trip_duration_and_manual_calc_time',df["trip_duration"].cast("double") - df['diff_in_dropff_and_pickup'].cast("double"))

In [12]:
# Checking for unique values in `diff_in_trip_duration_and_manual_calc_time`

df.select('diff_in_trip_duration_and_manual_calc_time').distinct().collect()

[Row(diff_in_trip_duration_and_manual_calc_time=0.0)]

In [13]:
# Checking if trip duration is 0 or negative, if there then we can remove it.

df.filter(df.trip_duration <= 0).count()

0

In [14]:
# Checking for unique values in `store_and_fwd_flag`

df.select('store_and_fwd_flag').distinct().collect()

[Row(store_and_fwd_flag=1), Row(store_and_fwd_flag=0)]

In [15]:
# Calculate distance between two latitute and longtitude.

def distance(pickUpLat1, dropOffLat2, pickUpLon1, dropOffLon2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(pickUpLon1)
    lon2 = radians(dropOffLon2)
    lat1 = radians(pickUpLat1)
    lat2 = radians(dropOffLat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
      
    # calculate the result
    return(c * r)

In [16]:
# Calculating distance between pickup and dropoff latitude and longtitude.

udf_func = udf(distance, FloatType())
df = df.withColumn("distance", udf_func(df.pickup_latitude, df.dropoff_latitude, df.pickup_longitude, df.dropoff_longitude))

In [17]:
# distance is in K.M.

df.select("distance").show()

+----------+
|  distance|
+----------+
| 1.4985207|
| 1.8055072|
| 6.3850985|
| 1.4854984|
| 1.1885885|
| 1.0989425|
| 1.3262786|
| 5.7149806|
| 1.3103533|
| 5.1211615|
| 3.8061395|
| 3.7730958|
|  1.859483|
|0.99168485|
|  6.382836|
|  0.656578|
|  3.428086|
| 2.5386717|
| 4.6052012|
| 1.3032712|
+----------+
only showing top 20 rows



In [18]:
# Only keep records where distance between non overlapped points

print("Count unique values Before filtering for distance < 0 is ", df.count()) 
df = df.filter(df.distance > 0)

Count unique values Before filtering for distance < 0 is  1458644


In [19]:
# Counting total number of unique values in dataframe.

df.count()

1452747

In [20]:
import os

if os.path.isdir('train-cleaned') == False:
    df.coalesce(1).write.csv("train-cleaned", header=True)